In [23]:
import os
import pandas as pd
import numpy as np
import re
from collections import Counter

species = 'abiotrophia_defectiva' 

# set directories
basedir = os.getcwd() 
comparisondir = basedir+'/'+species+'/Comparison'
diamond_directory=comparisondir+'/GFs/diamond/'


In [24]:
files=os.listdir(diamond_directory)

count = 0
empty = 0
div = 0
df_gfs_l = []
for f in files:
    delos2uniprot = dict()
    delos2scores = dict()
    
    try:
        pandas_df = pd.read_csv(diamond_directory+f,sep='\t',header=None)
    except:
        empty += 1
        df_gfs_l.append({'GFs':f.split('.')[0],
                     'mapping_Dim':0,
                     'Pan':0,
                     'Pan_prots':0,
                     'Pan_prot':0,
                     'Gen':0,
                     'Gen_prots':0,
                     'Gen_prot':0,
                     'Same':False})
        continue
    
    set_genes = set()
    for r in pandas_df.index:
        pandelos_gene = pandas_df.iloc[r,0]
        set_genes.add(pandelos_gene)
        uniprot_gene = pandas_df.iloc[r,1]

        score = pandas_df.iloc[r,2]
        if pandelos_gene in delos2scores:
            if delos2scores[pandelos_gene] > score:
                continue
            else:
                delos2scores[pandelos_gene] = score
                delos2uniprot[pandelos_gene] = uniprot_gene
        else:
            delos2uniprot[pandelos_gene] = uniprot_gene
            delos2scores[pandelos_gene] = score
            
    missing_map = set_genes - set(delos2uniprot.keys()) 
    if len(missing_map) >0 :
        print('missing: ',missing_map)
    
    r = re.compile(".*PANSPECIFIC")
    matching_gene = list(filter(r.match, delos2uniprot.keys()))
    panspecific_uniprot = [delos2uniprot[u] for u in matching_gene]
 
    r = re.compile(".*GENERAL")
    unmatching_gene = list(filter(r.match, delos2uniprot.keys()))
    general_uniprot = [delos2uniprot[u] for u in unmatching_gene]
    
    if len(general_uniprot) > 0:
        max_gen = max(Counter(general_uniprot))
    else:
        max_gen = 0
    if len(panspecific_uniprot) > 0:
        max_pan = max(Counter(panspecific_uniprot))
    else:
        max_pan = 0 
     
    flag = max_pan == max_gen
    if flag:
        count += 1
    else:         
        div += 1
        
    df_gfs_l.append({'GFs':f.split('.')[0],
                     'mapping_Dim':len(set_genes),
                     'Pan':len(panspecific_uniprot),
                     'Pan_prots':panspecific_uniprot,
                     'Pan_prot':max_pan,
                     'Gen':len(general_uniprot),
                     'Gen_prots':general_uniprot,
                     'Gen_prot':max_gen,
                     'Same':flag})

res=pd.DataFrame(df_gfs_l)

print('Matching function:',count)
print('Unmatching function:', div)
print('Protein unmapped:', empty)

print('Percentage Matching:',(count*100)/(div+empty+count))
print('Percentage Other:', ((div+empty)*100)/(div+empty+count))

Matching function: 800
Unmatching function: 25
Protein unmapped: 12
Percentage Matching: 95.57945041816009
Percentage Other: 4.4205495818399045


In [25]:
res

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same
0,gatA_1,16,1,[KIFDJALJ_00447],KIFDJALJ_00447,15,"[KIFDJALJ_00447, KIFDJALJ_00447, KIFDJALJ_0044...",KIFDJALJ_00447,True
1,nrdI,16,1,[KIFDJALJ_01241],KIFDJALJ_01241,15,"[KIFDJALJ_01241, KIFDJALJ_01241, KIFDJALJ_0124...",KIFDJALJ_01241,True
2,group_1130,14,1,[KIFDJALJ_01055],KIFDJALJ_01055,13,"[KIFDJALJ_01055, KIFDJALJ_01055, KIFDJALJ_0105...",KIFDJALJ_01055,True
3,group_1911,13,2,"[KIFDJALJ_00601, KIFDJALJ_00601]",KIFDJALJ_00601,11,"[KIFDJALJ_00601, KIFDJALJ_00601, KIFDJALJ_0060...",KIFDJALJ_00601,True
4,group_1431,15,1,[KIFDJALJ_01669],KIFDJALJ_01669,14,"[KIFDJALJ_01669, KIFDJALJ_01669, KIFDJALJ_0166...",KIFDJALJ_01669,True
...,...,...,...,...,...,...,...,...,...
832,iolU,16,1,[KIFDJALJ_01620],KIFDJALJ_01620,15,"[KIFDJALJ_01620, KIFDJALJ_01620, KIFDJALJ_0162...",KIFDJALJ_01620,True
833,coaBC,7,1,[KIFDJALJ_00653],KIFDJALJ_00653,6,"[KIFDJALJ_00653, KIFDJALJ_00653, KIFDJALJ_0065...",KIFDJALJ_00653,True
834,group_1984,13,1,[KIFDJALJ_00599],KIFDJALJ_00599,12,"[KIFDJALJ_00599, KIFDJALJ_00599, KIFDJALJ_0059...",KIFDJALJ_00599,True
835,ansB,9,2,"[KIFDJALJ_01419, KIFDJALJ_01419]",KIFDJALJ_01419,7,"[KIFDJALJ_01419, KIFDJALJ_01419, KIFDJALJ_0141...",KIFDJALJ_01419,True


In [26]:
#Add uniprot description
ref_genome =  os.listdir(basedir+'/'+species+"/input/complete")
ref_genome_m = [not r.endswith('.fai') for r in ref_genome]
ref_genome = np.array(ref_genome)[ref_genome_m][0]
print(ref_genome)
refseq_d=basedir+'/'+species+'/Roary/prokka/prokka_'+ref_genome
refseq_files=os.listdir(refseq_d)
refseq = [f for f in refseq_files if f.endswith('.faa')][0]
protacc2name = dict()
with open(refseq_d+'/'+refseq) as f:
    lines = f.readlines()
    for l in lines:
        if l.startswith('>'):
            l = l.split(' ')
            protein_acc = l[0].strip('>')
            name = " ".join(l[1:len(l)-2])
            protacc2name[protein_acc] = name
protacc2name[0] = 'Not mapped'

GCF_013267415.1_ASM1326741v1_genomic.fna


In [27]:
# Add true GF sizes (regardless of whether they were mapped in diamond)
gf_size = pd.read_csv(comparisondir+'/comparison_GF_sizes.tsv')
true_size = dict(zip(gf_size.gene, gf_size.GF_size_pandelos))
true_size_roa = dict(zip(gf_size.gene, gf_size.GF_size_Roary))

function_pan = []
true_size_l = []
true_size_r = []
function_gen = []
for r in range(0,len(res['Pan_prot'])):
    acc_pan = res.loc[r,'Pan_prot']
    acc_gen = res.loc[r,'Gen_prot']
    name = res.loc[r,'GFs']
    
    true_size_l.append(true_size[name])
    true_size_r.append(true_size_roa[name])

    function_pan.append(protacc2name[acc_pan])
    function_gen.append(protacc2name[acc_gen])
    
res['Fun_Pan']=function_pan
res['Fun_Gen']=function_gen
res['True_size']=true_size_l
res['True_size_general']=true_size_r


In [28]:
res.to_csv(comparisondir+'/function.csv')
res

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same,Fun_Pan,Fun_Gen,True_size,True_size_general
0,gatA_1,16,1,[KIFDJALJ_00447],KIFDJALJ_00447,15,"[KIFDJALJ_00447, KIFDJALJ_00447, KIFDJALJ_0044...",KIFDJALJ_00447,True,PTS system galactitol-specific,PTS system galactitol-specific,16,15
1,nrdI,16,1,[KIFDJALJ_01241],KIFDJALJ_01241,15,"[KIFDJALJ_01241, KIFDJALJ_01241, KIFDJALJ_0124...",KIFDJALJ_01241,True,,,16,15
2,group_1130,14,1,[KIFDJALJ_01055],KIFDJALJ_01055,13,"[KIFDJALJ_01055, KIFDJALJ_01055, KIFDJALJ_0105...",KIFDJALJ_01055,True,,,14,13
3,group_1911,13,2,"[KIFDJALJ_00601, KIFDJALJ_00601]",KIFDJALJ_00601,11,"[KIFDJALJ_00601, KIFDJALJ_00601, KIFDJALJ_0060...",KIFDJALJ_00601,True,,,13,11
4,group_1431,15,1,[KIFDJALJ_01669],KIFDJALJ_01669,14,"[KIFDJALJ_01669, KIFDJALJ_01669, KIFDJALJ_0166...",KIFDJALJ_01669,True,,,15,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,iolU,16,1,[KIFDJALJ_01620],KIFDJALJ_01620,15,"[KIFDJALJ_01620, KIFDJALJ_01620, KIFDJALJ_0162...",KIFDJALJ_01620,True,scyllo-inositol 2-dehydrogenase,scyllo-inositol 2-dehydrogenase,17,16
833,coaBC,7,1,[KIFDJALJ_00653],KIFDJALJ_00653,6,"[KIFDJALJ_00653, KIFDJALJ_00653, KIFDJALJ_0065...",KIFDJALJ_00653,True,Coenzyme A biosynthesis bifunctional,Coenzyme A biosynthesis bifunctional,7,6
834,group_1984,13,1,[KIFDJALJ_00599],KIFDJALJ_00599,12,"[KIFDJALJ_00599, KIFDJALJ_00599, KIFDJALJ_0059...",KIFDJALJ_00599,True,,,13,12
835,ansB,9,2,"[KIFDJALJ_01419, KIFDJALJ_01419]",KIFDJALJ_01419,7,"[KIFDJALJ_01419, KIFDJALJ_01419, KIFDJALJ_0141...",KIFDJALJ_01419,True,,,9,7


In [29]:
res[res['Same']==False]

,GFs,mapping_Dim,Pan,Pan_prots,Pan_prot,Gen,Gen_prots,Gen_prot,Same,Fun_Pan,Fun_Gen,True_size,True_size_general
12,rpoC,13,0,[],0,13,"[KIFDJALJ_00883, KIFDJALJ_00883, KIFDJALJ_0088...",KIFDJALJ_00883,False,Not mapped,DNA-directed RNA polymerase,15,13
13,group_2350,8,8,"[KIFDJALJ_01169, KIFDJALJ_01169, KIFDJALJ_0116...",KIFDJALJ_01169,0,[],0,False,,Not mapped,8,1
69,yigZ,15,0,[],0,15,"[KIFDJALJ_00993, KIFDJALJ_00993, KIFDJALJ_0099...",KIFDJALJ_00993,False,Not mapped,IMPACT family,17,16
74,carB,13,0,[],0,13,"[KIFDJALJ_01887, KIFDJALJ_01887, KIFDJALJ_0188...",KIFDJALJ_01887,False,Not mapped,Carbamoyl-phosphate synthase,14,13
92,group_2315,0,0,0,0,0,0,0,False,Not mapped,Not mapped,4,1
129,murQ,10,0,[],0,10,"[KIFDJALJ_01195, KIFDJALJ_01195, KIFDJALJ_0119...",KIFDJALJ_01195,False,Not mapped,N-acetylmuramic acid,11,10
136,group_107,10,3,"[KIFDJALJ_01297, KIFDJALJ_01294, KIFDJALJ_01294]",KIFDJALJ_01297,7,"[KIFDJALJ_01300, KIFDJALJ_01300, KIFDJALJ_0130...",KIFDJALJ_01300,False,,,10,7
177,group_570,16,8,"[KIFDJALJ_00478, KIFDJALJ_00478, KIFDJALJ_0048...",KIFDJALJ_00487,8,"[KIFDJALJ_00478, KIFDJALJ_00478, KIFDJALJ_0047...",KIFDJALJ_00478,False,,,16,8
240,dnaA,9,0,[],0,9,"[KIFDJALJ_00818, KIFDJALJ_00818, KIFDJALJ_0081...",KIFDJALJ_00818,False,Not mapped,Chromosomal replication initiator,10,9
258,group_1505,10,7,"[KIFDJALJ_01213, KIFDJALJ_01213, KIFDJALJ_0091...",KIFDJALJ_01215,3,"[KIFDJALJ_01213, KIFDJALJ_01213, KIFDJALJ_01213]",KIFDJALJ_01213,False,Adaptive-response,,10,3
